# 【問題1】チャンネル数を1に限定した1次元畳み込み層クラスの作成

# 【問題2】1次元畳み込み後の出力サイズの計算


# 【問題3】小さな配列での1次元畳み込み層の実験


# 【問題4】チャンネル数を限定しない1次元畳み込み層クラスの作成

# 【問題5】学習・推定

In [23]:
#インポート
from tqdm import tqdm_notebook as tqdm
import pyprind
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from keras.datasets import fashion_mnist
from keras.datasets import mnist
from sklearn.preprocessing import OneHotEncoder
%matplotlib inline

Using TensorFlow backend.


In [10]:
class Conv1d:
    """
    1次元畳み込み層クラス
    2次元にも対応可能
    
    Parameters
    ----------
    FN : int
      出力チャンネル数（フィルターの個数）
    FH : int
      フィルターの高さ
    FW : int
      フィルターの幅
    S : int (default: 1)
      ストライド
    P : int (default: 0)
      パディング

    Attributes
    ----------
    group : 'conv'
      layerの種類
    
    """

    def __init__(self,
                 FN=3,
                 FH=3,
                 FW=3,
                 P=0,
                 S=1):
        # 初期化
        # initializerのメソッドを使い、self.Wとself.Bを初期化する           
        self.FH = FH
        self.FW = FW
        self.FN = FN
        self.P = P
        self.S = S
        self.group = 'conv'
        
    def initialize(self, input_dim,summary, init_type, optimizer, sigma=1e-2, lr=1e-2):
        """
        W,B,initializer,optimizerを初期化するメソッド
        
        Parameters
        ----------        
        input_dim :次の形のtuple, (入力チャンネル,高さ,横幅)
            入力サイズ
        summary: bool
            Trueにするとshapeを出力
        initializer: class 
            initializerのクラス
        optimizer: 
            optimizerのクラス
        sigma:float
            Simpleinitializerを選んだ時のパラメータ
            
        Return
        ----------
        out_dim:次の形のtuple, (出力チャンネル,OH,OW)
        """
        #出力サイズを計算する。
        C, H, W = input_dim
        OH = out_calc(H, self.P, self.FH, self.S)
        OW = out_calc(W, self.P, self.FW, self.S)
        out_dim = (self.FN, OH, OW)
        
        #初期値を設定する。
        initializer = Initializer(init_type, C * self.FH * self.FW, sigma)
        self.W = initializer.W(self.FN, C, self.FH, self.FW)
        self.B = initializer.B(self.FN)
        if summary:
            print(self.group,'layer shape={}, param={}'.format(out_dim, self.FH*self.FW*C*self.FN+self.FN))
                
        #optimizerを設定する。
        self.optimizer = optimizer(lr)
        return out_dim 

    def forward(self, X):
        """
        フォワード
        Parameters
        ----------
        X : 次の形のndarray, shape (N,C,高さ,横幅)
            入力値
        W: 次の形のndarray, shape (FN,C,FH,FW)
            重み

        Returns
        ----------
        out : 次の形のndarray, shape (N、FN,OH, OW)
            出力
        """
        X_copy = X.copy()
        self.X_shape = X_copy.shape #バックワードで使うのでshapeを保存しておく。
        self.X_out = im2patch(X_copy, self.FH, self.FW, self.S, self.P)  
        # X_out.shape(N, C, OH(dAをかける), OW(dAをかける), input_h(Wをかける), input_w(Wをかける))
        out = np.tensordot(self.X_out, self.W, axes=[[1, 4, 5], [1, 2, 3]]) 
        out = out.transpose([0, 3, 1, 2])  # (N、FN,OH, OW) の形に直す
        out += self.B[np.newaxis, :, np.newaxis, np.newaxis] #FN分だけBをブロードキャストして足す。

        return out

    def backward(self, dA):
        """
        バックワード
        Parameters
        ----------
        dA : 次の形のndarray, shape (N、FN,OH, OW)
            後ろから流れてきた勾配
        Returns
        ----------
        dW: 次の形のndarray, shape (FN,C,FH,FW)
            前に流すWの勾配
        dB : 次の形のndarray, shape (N,FN)
            前に流すBの勾配
        dX : 次の形のndarray, shape (N,C,高さ,横幅)
            前に流すWの勾配

        """
        #勾配を計算
        self.dW = np.tensordot(dA, self.X_out, axes=[[0, 2, 3], [0, 2,3]])  #N,OH,OWの軸でテンソルドットする。
        self.dB = np.sum(dA, axis=(-1, -2))  # (N、FN)を残してsum
        # 出力を計算
        out = np.tensordot(dA, self.W, axes=[1, 0])#FNの軸でdot積
        out = out.transpose(0, 3, 1,2, 4, 5)
        # out.shape(N,C,OH, OW, FH, FW)
        dX = patch2im(out, self.X_shape, self.S, self.P)

        # 更新
        self = self.optimizer.update(self)

        return dX

In [11]:
def out_calc(n_in, P=0, FS=3, S=1):
    """
    出力サイズの計算。1次元。
    n_in: int
        特徴量の数
    p: int(0)
        パディングの数
    FS:int(3)
        フィルタのサイズ
    s: int(1)
        ストライドのサイズ        
    """
    return int(((n_in + 2 * P - FS) / S) + 1)

In [12]:
def im2patch(input_data, FH, FW, S=1, P=0):
    """
    ストライドした後の配列を出してくれる関数

    Parameters
    ----------
    input_data : (データ数, チャンネル, 高さ, 幅)の4次元配列からなる入力データ
    FH : フィルターの高さ
    FW : フィルターの幅
    S : ストライド
    P : パディング

    Returns
    -------
    patch : shape(データ数, チャンネル, OH(dAをかける), OW(dAをかける), FH(Wをかける), FW(Wをかける)) 
    """
    N, C, H, W = input_data.shape
    OH = out_calc(H, P, FH, S)
    OW = out_calc(W, P, FW, S)
    img = np.pad(input_data, [(0, 0), (0, 0), (P, P), (P, P)],'constant')
    patch = np.empty((N, C, OH, OW, FH, FW))
    #出力サイズ(OH,OW)の軸を固定してフィルター(FH,HW)を掛けた部分を代入していく
    for y in range(OH):
        IH = y * S
        IH_max = IH + FH
        for x in range(OW):
            IW = x * S
            IW_max = IW + FW
            patch[:, :, y, x, :, :] = img[:, :, IH:IH_max, IW:IW_max]
    return patch


def patch2im(input_data, input_shape, S=1, P=0):
    """
    ストライドした後の配列を出してくれる関数

    Parameters
    ----------
    input_data : (データ数,チャンネル数, OH, OW,FH, FW)の6次元配列からなる入力データ
    input_shape:(データ数,チャンネル数、高さ、横幅)
    S : ストライド
    P : パディング

    Returns
    -------
    back
    out : shape(データ数, チャンネル, IH, IW)
    """
    N, C, H, W = input_shape
    _, _,  OH, OW, FH, FW = input_data.shape
    im = np.empty((N, C, H + 2 * P, W + 2 * P ))
    # im2patchと逆のことをする。
    for y in range(OH):
        IH = y * S
        IH_max = IH + FH 
        for x in range(OW):
            IW = x * S
            IW_max = IW + FW
            im[:, :, IH:IH_max, IW:IW_max] += input_data[:, :, y, x, :, :]

    return im[:, :, P:P + H, P:P + W]

In [13]:
class GetMiniBatch:
    """
    ミニバッチを取得するイテレータ

    Parameters
    ----------
    X : 次の形のndarray, shape (n_samples, n_features)
      学習データ
    y : 次の形のndarray, shape (n_samples, 1)
      正解値
    batch_size : int
      バッチサイズ
    """
    def __init__(self, X, y, batch_size = 10):
        self.batch_size = batch_size
        shuffle_index = np.random.permutation(np.arange(X.shape[0]))
        self.X = X[shuffle_index]
        self.y = y[shuffle_index]
        self._stop = np.ceil(X.shape[0]/self.batch_size).astype(np.int)

    def __len__(self):
        return self._stop

    def __getitem__(self,item):
        p0 = item*self.batch_size
        p1 = item*self.batch_size + self.batch_size
        return self.X[p0:p1], self.y[p0:p1]        

    def __iter__(self):
        self._counter = 0
        return self

    def __next__(self):
        if self._counter >= self._stop:
            raise StopIteration()
        p0 = self._counter*self.batch_size
        p1 = self._counter*self.batch_size + self.batch_size
        self._counter += 1
        return self.X[p0:p1], self.y[p0:p1]

In [14]:
class FC:
    """
    ノード数pre_nodesからnodesへの全結合層
    Parameters
    ----------
    nodes : int
      層のノード数
    
    Attributes
    ----------
    group : 'FC'
      layerの種類
    """
    def __init__(self, nodes):
        self.nodes = nodes
        self.group = 'FC'
        
    def initialize(self, pre_nodes, summary, init_type, optimizer, sigma=1e-2, lr=1e-2):
        """
        重み、バイアスを初期化して出力数を渡してあげる
        Parameters
        ----------
        input_dim :次の形のtuple, (入力チャンネル,高さ,横幅)
            入力サイズ
        initializer: class
            initializerのクラス
        optimizer: class
            optimizerのクラス
        lr : float(1e-2)
            optimizerに渡す学習率
        sigma : float(1e-2)
            Simpleinitializerを選んだ時のパラメータ
        """
        
        #初期値を設定する。
        initializer = Initializer(init_type, pre_nodes, sigma)
        self.W = initializer.W(pre_nodes, self.nodes)
        self.B = initializer.B(self.nodes)
        
        if summary:
            print(self.group,'layer shape={}, param={}'.format(self.W.shape,pre_nodes*self.nodes+self.nodes))

        #optimizerを設定する。
        self.optimizer = optimizer(lr)
        
        return self.nodes

    def forward(self, X):
        """
        フォワード
        Parameters
        ----------
        X : 次の形のndarray, shape (batch_size, pre_nodes)
            入力
        Returns
        ----------
        A : 次の形のndarray, shape (batch_size, nodes)
            出力
        """        
        self.Z = X.copy()
        A = X @ self.W + self.B
        return A
    def backward(self, dA):
        """
        バックワード
        Parameters
        ----------
        dA : 次の形のndarray, shape (batch_size, nodes)
            後ろから流れてきた勾配
        Returns
        ----------
        dZ : 次の形のndarray, shape (batch_size, pre_nodes)
            前に流す勾配
        """
        self.dB = dA
        self.dW = self.Z.T @ dA
        dZ = dA @ self.W.T
        # 重みを更新
        self = self.optimizer.update(self)
        return dZ

In [16]:
class SGD:
    """
    確率的勾配降下法
    Parameters
    ----------
    lr : 学習率
    """
    def __init__(self, lr):
        self.lr = lr
    def update(self, layer):
        """
        ある層の重みやバイアスの更新
        Parameters
        ----------
        layer : 更新前の層のインスタンス

        Returns
        ----------
        layer : 更新後の層のインスタンス
        """
        
        layer.W -= self.lr * (layer.dW / layer.dB.shape[0])
        layer.B -= self.lr * np.mean(layer.dB, axis=0)
        return layer
    
class AdaGrad:
    """
    AdaGrad
    Parameters
    ----------
    lr : 学習率
    """
    def __init__(self, lr):
        self.lr = lr
        self.H_w = 0 #最初は0
        self.H_b = 0
    def update(self, layer):
        """
        ある層の重みやバイアスの更新
        Parameters
        ----------
        layer : 更新前の層のインスタンス

        Returns
        ----------
        layer : 更新後の層のインスタンス
        """
        dW_mean = layer.dW / layer.dB.shape[0] #行列
        dB_mean = np.mean(layer.dB, axis=0)
        #重みの更新
        self.H_w += dW_mean * dW_mean # Hを更新
        layer.W -= self.lr * dW_mean / (np.sqrt(self.H_w) + 1e-7)
        
        self.H_b +=  dB_mean * dB_mean        
        layer.B -= self.lr * dB_mean / (np.sqrt(self.H_b) + 1e-7)
        
        return layer

In [17]:
class Sigmoid:
    """
    　シグモイド関数の活性化関数
    Parameters
    ----------
    """
    def __init__(self):
        self.Z = None
        self.group = 'activation'
    def forward(self, A):
        """
        フォワードプロパゲーションのときのメソッド
        Parameters
        ----------
        A : 全結合後の行列 shapeはどんな形でも大丈夫

        Returns
        ----------
        Z : 活性化後の行列　元のshapeを保持
        """
        self.Z = 1 / (1 + np.exp(-A))
        return self.Z
    
    def backward(self, dZ):
        """
        バックワード
        Parameters
        ----------
        dZ : 全結合後の行列  shapeはどんな形でも大丈夫

        Returns
        ----------
        dA : 活性化後の行列　元のshapeを保持
        """
        dA = dZ * (1 - self.Z) * self.Z
        return dA

class Tanh:
    """
    ハイパボリックタンジェント関数
    Parameters
    ----------
    """
    def __init__(self):
        self.Z = None
        self.group = 'activation'

    def forward(self, A):
        """
        フォワードプロパゲーションのときのメソッド
        Parameters
        ----------
        A : 全結合後の行列 shapeはどんな形でも大丈夫

        Returns
        ----------
        Z : 活性化後の行列 元のshapeを保持
        """
        self.Z = np.tanh(A)
        return self.Z
    
    def backward(self, dZ):
        """
        バックワード
        Parameters
        ----------
        dZ : 全結合後の行列 shapeはどんな形でも大丈夫

        Returns
        ----------
        dA : 活性化後の行列　元のshapeを保持
        """
        dA = dZ * (1 - self.Z ** 2)
        return dA
    
class Relu:
    """
    ReLU関数の活性化関数
    Parameters
    ----------
    """
    def __init__(self):
        self.group = 'activation'
        self.mask = None

    def forward(self, A):
        """
        フォワードプロパゲーションのときのメソッド
        Parameters
        ----------
        A : 全結合後の行列 shapeはどんな形でも大丈夫

        Returns
        ----------
        Z : 活性化後の行列　元のshapeを保持
        """
        self.mask = A <= 0
        Z = A.copy()
        Z[self.mask] = 0
        return  Z

    
    def backward(self, dZ):
        """
        バックワード
        Parameters
        ----------
        dZ : 全結合後の行列shapeはどんな形でも大丈夫

        Returns
        ----------
        dA : 活性化後の行列　元のshapeを保持
        """
        dZ[self.mask] = 0
        
        return dZ
    
class Softmax:
    """
    ソフトマックス関数の活性化関数
    Parameters
    ----------
    """
    def __init__(self):
        self.Z = None
        self.entropy = None # バッチ単位でのエントロピー
        self.group = 'activation'
        
    def forward(self, A):
        """
        フォワードプロパゲーションのときのメソッド
        Parameters
        ----------
        A : 全結合後の行列 shape(batch_size, pre_nodes)

        Returns
        ----------
        Z : 活性化後の行列　shape(batch_size,  pre_nodes)
        """
        c = np.max(A, axis=1,keepdims=True)
        self.Z = np.exp(A-c) / np.sum(np.exp(A-c), axis=1).reshape(-1, 1)
        return self.Z
    
    def backward(self, Y):
        """
        バックワードと交差エントロピーを計算
        Parameters
        ----------
        dZ : 全結合後の行列 shape(batch_size, pre_nodes)

        Returns
        ----------
        dA : 活性化後の行列　shape(batch_size,  pre_nodes)
        """
        entropy = -np.sum(Y * np.log(self.Z + 1e-5), axis=1) #サンプル毎のエントロピー(batch_size,)
        self.entropy = entropy.sum() / len(entropy)  # スカラー
        #勾配はこっち
        dA = self.Z - Y
        return dA

In [18]:
class Initializer:
    """
    ガウス分布によるシンプルな初期値設定
    Parameters
    ----------
    sigma : float
      ガウス分布の標準偏差
    """
    def __init__(self, init_type, pre_nodes, sigma):
        if init_type == 'simple':
            self.sigma = sigma
        elif init_type == 'Xavier':
            self.sigma = 1 / np.sqrt(pre_nodes)
        elif init_type == 'He':
            self.sigma = np.sqrt(2 / pre_nodes)

    def W(self,*args):
        """
        重みの初期化
        Parameters
        ----------
        args : int
          ノード数や、チャンネル数等必要なサイズを入力
        Returns
        ----------
        W :次の形のndarray, shape (args)
            重み
        """
        W = self.sigma * np.random.standard_normal(size=args)
        return W
    def B(self, *args):
        """
        バイアスの初期化
        Parameters
        ----------
        args : int
          ノード数等を入力。入力した形の必要なサイズを入力
        Returns
        ----------
        B :次の形のndarray, shape (args)
            バイアス
        """
        B = self.sigma * np.random.standard_normal(size=args)
        return B

In [19]:
class Flatten:
    """
    平滑化するクラス
        
    Parameters
    ----------

    Attributes
    ----------
    group : 'flatten'
      layerの種類
    """
    def __init__(self):
        # 初期化    
        self.group = 'flatten'
        
    def initialize(self, input_dim,summary, *args, **kargs):
        """
        出力サイズを出力するメソッド
        [*args, **kargs]はダミー    
        Return
        ----------
        out_dim:次の形のtuple, (出力チャンネル,OH,OW)
        """
        #出力サイズを計算する。
        C, H, W = input_dim
        if summary:
            print(self.group,'layer shape=', (C*H*W))

        return C*H*W

        
    def forward(self, X):
        """
        フォワード
        Parameters
        ----------
        X : 次の形のndarray, shape (N,C,高さ,横幅)

        Returns
        ----------
        A : 次の形のndarray, shape (N、C*高さ*横幅)
            出力
        """
        self.X_shape = X.shape #バックワードで使用
        out = X.reshape(len(X), -1) #サンプルサイズだけ残してflatへ
        return out
    
    def backward(self, dout):
        """
        バックワード
        Parameters
        ----------
        dout: 次の形のndarray, shape(N, n_nodes)

        Returns
        ----------
        A : 次の形のndarray, shape (N、C,高さ,横幅)
            出力
        """
                
        dout = dout.reshape(self.X_shape)
        return dout
        

In [21]:
class Scratch1dCNNClassifier:
    """
    ディープなニューラルネットワーク分類器
    層を増やすことが出来る。
    バッチをランダムで抽出する。
    エポック毎にバッチを取り直すことも可能。
    2Dにも対応可能
    Parameters
    ----------
    batch_size : int　(30)
        バッチサイズ
    n_epoch : int (100)
        エポック数
    e_threshold : float(1e-2)
        エポック途中終了の為のエントロピーの閾値
    n_iter : int(1000)
        1エポック辺りのイテレーション数
    repeat_batch_process : bool(True)
        Trueの場合１エポック毎にバッチをランダムに取り直す。
    restore_extraction:bool(True)
        学習するバッチをランダム抽出する際に復元か、非復元か選ぶ。基本は復元(ブートストラップ)
    seed : int(0)
        ランダムシード
        
    Attributes
    ----------
    entropy : shape(n_epoch, n_iter)
        1バッチごとのエントロピー
    layers : list
        layerのリスト
    
    """

    def __init__(
        self,
        batch_size=30,
        n_epochs=100,
        e_threshold=1e-2,
        n_iter=1000,
        repeat_batch_process=True,
        restore_extraction=True,
        seed=0
    ):
        self._n_iter = n_iter
        self._repeat_batch_process = repeat_batch_process
        self._restore_extraction = restore_extraction
        self._batch_size = batch_size
        self._n_epochs = n_epochs
        self._e_threshold = e_threshold  # 誤差の閾値        
        self.entropy = None
        self.epoch_entropy_mean = None
                            
    def sequential(self,*layers):
        """
        layerをつなげるメソッド。
        """
        self.layers = []        
        for layer in layers:
            self.layers.append(layer)
            
    def initialize(self, input_dim, summary, init_type, optimizer, sigma=1e-2, lr=1e-2):
        """
        それぞれのlayerの初期化メソッド
        活性化層以外の層のinitializeメソッドを使う。
        """
        for layer in self.layers:
            if layer.group != 'activation':
                input_dim = layer.initialize(input_dim,summary, init_type, optimizer, sigma=sigma, lr=lr)
        

    def fit(self, X, y):
        """
        NNを学習する。

        Parameters
        ----------
        X : 次の形のndarray, shape (n_samples, チャンネル数,高さ,幅)
            学習用データの特徴量
        y : 次の形のndarray, shape (n_samples, クラス)
            学習用データの正解値(one_hot_vectaされた後のもの)
        
        """

        # リピートしない場合はここでミニバッチ化
        if self._repeat_batch_process == False:
            train_batch = GetMiniBatch(X, y,
                                       batch_size=self._batch_size) 
        # バッチをランダム取得するためのindexを取得
        batch_index = np.random.choice(int(len(X) / self._batch_size),
                                       self._n_iter,
                                       replace=self._restore_extraction)  # n_iterのindex
        self.entropy = np.zeros(
            (self._n_epochs, self._n_iter))  # エポック数×イテレーション数のエントロピーの入れ物

        # 学習開始
        for epoch in tqdm(range(self._n_epochs)):
            if self._repeat_batch_process:
                train_batch = GetMiniBatch(
                    X, y, batch_size=self._batch_size)  # バッチに分ける。
            batch_entropy = np.zeros(self._n_iter)  # 1エポック内でのエントロピーを格納。
            for i, index in enumerate(batch_index):
                X_batch, y_batch = train_batch[index][0].copy(
                ), train_batch[index][1].copy()
                # フォワードプロパゲーション
                for layer_index in range(len(self.layers)):
                    X_batch = self.layers[layer_index].forward(X_batch)
                # バックプロパゲーション
                for layer_index in range(1, len(self.layers) + 1):
                    y_batch = self.layers[-layer_index].backward(y_batch)

                    # 誤差を格納
                batch_entropy[i] = self.layers[-1].entropy
                if i % 100 == 0: #100 バッチごとに状況を報告
                    print('{}回目のbatch_entropy= {:.5f}'.format(i+epoch*self._n_iter,batch_entropy[i]))

            self.entropy[epoch, :] = batch_entropy  # バッチごとのエントロピーを格納
            if batch_entropy.mean() < self._e_threshold:  # 誤差が閾値以下になったらエポック終了
                print('entropyが{:.3f}より低いよ！'.format(self._e_threshold))
                break
                
        #最後までエポックが回らなかった時に後ろの0を消去
        self.entropy = self.entropy[self.entropy != 0]

    def predict(self, X):
        """
        NNで予測する。

        Parameters
        ----------
        X : 次の形のndarray, shape (n_samples, チャンネル数,高さ,幅)
            学習用データの特徴量        
        """

        # フォワードプロパゲーション
        out = X.copy()
        for layer in range(len(self.layers)):
            out = self.layers[layer].forward(out)

        return np.argmax(out, axis=1)

In [24]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train = X_train / 255
X_test = X_test / 255

In [25]:
X_train = np.expand_dims(X_train, 1)
X_test = np.expand_dims(X_test, 1)
enc = OneHotEncoder(handle_unknown='ignore', sparse=False)
y_train_one_hot = enc.fit_transform(y_train[:, np.newaxis])

In [26]:
cnn1d = Scratch1dCNNClassifier(batch_size=100,
                                n_epochs=5,
                                n_iter=500,
                                repeat_batch_process=False,
                                restore_extraction=False,
                                seed=0)

In [27]:
cnn1d.sequential(
    Conv1d(FN=6,FH=3, FW=3,  P=2, S=1), 
    Relu(),
    Flatten(), 
    FC(128),
    Relu(),
    FC(50), 
    Relu(),
    FC(10), 
    Softmax()
)

input_dim = (1, 28, 28)
cnn1d.initialize(input_dim, summary=True, init_type='He', optimizer=AdaGrad, lr=1e-2)

conv layer shape=(6, 30, 30), param=60
flatten layer shape= 5400
FC layer shape=(5400, 128), param=691328
FC layer shape=(128, 50), param=6450
FC layer shape=(50, 10), param=510


In [28]:
cnn1d.fit(X_train, y_train_one_hot)

0回目のbatch_entropy= 2.57254
100回目のbatch_entropy= 0.62507
200回目のbatch_entropy= 0.42283
300回目のbatch_entropy= 0.39267
400回目のbatch_entropy= 0.38123
500回目のbatch_entropy= 0.37014
600回目のbatch_entropy= 0.30933
700回目のbatch_entropy= 0.22057
800回目のbatch_entropy= 0.24994
900回目のbatch_entropy= 0.30004
1000回目のbatch_entropy= 0.30345
1100回目のbatch_entropy= 0.24930
1200回目のbatch_entropy= 0.17263
1300回目のbatch_entropy= 0.19195
1400回目のbatch_entropy= 0.25865
1500回目のbatch_entropy= 0.27347
1600回目のbatch_entropy= 0.22414
1700回目のbatch_entropy= 0.14656
1800回目のbatch_entropy= 0.15867
1900回目のbatch_entropy= 0.23502
2000回目のbatch_entropy= 0.25301
2100回目のbatch_entropy= 0.20447
2200回目のbatch_entropy= 0.12859
2300回目のbatch_entropy= 0.13901
2400回目のbatch_entropy= 0.21894



In [29]:
(cnn1d.predict(X_test) == y_test).sum() / len(X_test)

0.9561